In [1]:
import ehm_dmrg.ehm_dmrg as ehm
import ehm_dmrg
import pandas as pd
from pathlib import Path
import importlib
import ehm_dmrg.dmrg_performance_metrics as dpm
import json
import time


In [2]:
calc_resistance_matrix_properties = True
fcidump_location = "/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts"
# json_feature_storage = "/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/feature_store/initial_catalysts_plus_small.json"
json_feature_storage = "/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/feature_store/expanded_ehm.json"


# Location of csv files
# datafiles_folder = "/mnt/d/ZapataCalcs/GSEE/DMRG/DMRG_output_data"
datafiles_folder = "/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/performance_metrics_store"
# datafiles_folder = "../"
# datafile_name = "hami_9_data.csv"

# fcidump_files =["fcidump.2_co2_6-311++G__","fcidump.36_1ru_II_2pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }"]
fcidump_files = [
    "fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.2_co2_6-311++G__",
    "fcidump.3_ts_ru_macho_co2_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.5_ts_ru_macho_melact_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.7_melact_6-311++G__",
    "fcidump.8_melact_6-311++G__",
    "fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}",
    "fcidump.10_fecp2+_s0.5_def2-tzvp",
    "fcidump.11_fecp2_s0_def2-tzvp",
    "fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}",
    # "fcidump.16_ts_1over4a_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p", # min ΔE = 0.003510686596200685
    # "fcidump.18_I_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", # min ΔE = 0.008188852631519694
    "fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.22_pc-_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)",
    "fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)",
    "fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.28_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.29_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.30_4a_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.31_4a_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.32_2ru_III_3pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.33_2ru_III_3pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.34_3ruo_IV_2pl_{'Ru' _ 'lanl2tz', 'default' _ '6-31+G(d,p)'}",
    "fcidump.35_3ruo_IV_2pl_{'Ru' _ 'lanl2tz', 'default' _ '6-31+G(d,p)'}",
    "fcidump.36_1ru_II_2pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.37_1ru_II_2pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.38_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.39_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.40_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.41_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.43_1_star_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.45_1_star_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.46_2_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.47_2_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.48_2_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.49_2_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.50_6acme_{'default' _ '6-31++G(d,p)'}",
    "fcidump.51_6acme_{'default' _ '6-31++G(d,p)'}",
    "fcidump.53_ts56_{'default' _ '6-31++G(d,p)'}",
    "fcidump.54_ts56_{'default' _ '6-31++G(d,p)'}",
    "fcidump.55_ts56_{'default' _ '6-31++G(d,p)'}",
    "fcidump.56_5_{'default' _ '6-31++G(d,p)'}",
    "fcidump.57_5_{'default' _ '6-31++G(d,p)'}",
    "fcidump.58_5_{'default' _ '6-31++G(d,p)'}",
    "fcidump.59_5_16_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.60_5_16_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.61_3_15_af_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.62_3_15_af_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.63_5_15_af_ts_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
]

In [3]:
temp_dict = {}
for fcidump_file in fcidump_files:
    mol_index = fcidump_file.split("_")[0].split(".")[1]
    temp_dict[mol_index] = fcidump_file

fcidump_files_temp = []
for key in sorted(temp_dict.keys()):
    fcidump_files_temp.append(temp_dict[key])
print(fcidump_files_temp)

["fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", "fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", 'fcidump.10_fecp2+_s0.5_def2-tzvp', 'fcidump.11_fecp2_s0_def2-tzvp', "fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}", 'fcidump.2_co2_6-311++G__', "fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.22_pc-_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)", "fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)", "fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidu

In [4]:
# features_dataframe =
fcidump_location = Path(fcidump_location)

importlib.reload(ehm)
importlib.reload(ehm_dmrg)

dmrg_features_molecules_list_store = []
# load already calculated features
if Path(json_feature_storage).exists():
    with open(json_feature_storage, "r") as file:
        dmrg_features_molecules_list_store = file.readlines()
    dmrg_features_molecules_list_store = [json.loads(x) for x in dmrg_features_molecules_list_store]
    extant_hami_ids = [x["hamiltonian_id"] for x in dmrg_features_molecules_list_store]
else:
    extant_hami_ids = []

start_time = time.perf_counter_ns()
dmrg_features_molecules_list = []
for fcidump_file in fcidump_files:
    loop_start_time = time.perf_counter_ns()
    fcidump_filepath = fcidump_location / Path(fcidump_file)
    print(fcidump_filepath)

    if fcidump_file in extant_hami_ids:
        print(f"Skipping {fcidump_file} as it already exists in the feature store")
        extant_hami_ids_index = extant_hami_ids.index(fcidump_file)
        dmrg_features_molecules_list.append(dmrg_features_molecules_list_store[extant_hami_ids_index])
        assert dmrg_features_molecules_list[-1]["hamiltonian_id"] == fcidump_file
        print(f"Loaded {fcidump_file} data from the feature store.")
        continue
    
    dmrg_features_one_molecule={}
    dmrg_features_one_molecule["hamiltonian_id"] = fcidump_file

    dmrg_features_one_molecule_new = ehm.dmrg_features_from_fcidump(
        fcidump_filepath=fcidump_filepath,
        MOLPRO_ORBSYM=True,
        calc_resistance_matrix_properties=calc_resistance_matrix_properties,
        verbosity=1,
    )
    dmrg_features_one_molecule.update(dmrg_features_one_molecule_new)
    print(dmrg_features_one_molecule)
    
    with open(json_feature_storage, "a") as file:
            file.write(json.dumps(dmrg_features_one_molecule))  # use `json.loads` to do the reverse
            file.write("\n")
    dmrg_features_molecules_list.append(dmrg_features_one_molecule)
    current_time = time.perf_counter_ns()
    loop_time = current_time - loop_start_time
    
    print(f"Loop time (s) for {fcidump_file} is {loop_time/1e9}")
current_time = time.perf_counter_ns()    
print(f"Total time (s) for all molecules is {(current_time - start_time)/1e9}")

/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}
Parsing /mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}
meta data features: {'num_orbitals': 16, 'num_electrons': 18, 'num_spin_orbitals': 32, 'two_S': 0, 'orbsym': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'isym': 1}
one_body_tensor.shape (16, 16)
two_body_tensor.shape (16, 16, 16, 16)
one_body_tensor.shape (16, 16)
two_body_tensor.shape (16, 16, 16, 16)
one_and_two_body_tensor.shape (16, 16, 16, 16)
num_orbitals 16
num_electrons 18
Generated tbt matrices, time: 3.7189e-05
coulomb_matrix.shape (16, 16)
exchange_matrix.shape (16, 16)
mean_field_matrix.shape (16, 16)
Bandwidth calculation time: 4.9492e-05
Coulomb matrix features calculated
Coulomb matrix calculation time: 0.101780907
Exchange matrix features calculated
Exchange matrix calculation time: 0.012424914
Mean field matrix features calculated
Mean field m

In [5]:
ml_features_df = pd.DataFrame(dmrg_features_molecules_list)
ml_features_df

,hamiltonian_id,num_orbitals,num_electrons,num_spin_orbitals,two_S,orbsym,isym,total_num_states,log10_hilbert_space_size,tbt_matrices_calc_time_s,...,l2_norm_diff_total,hubbard_distance_calc_time_s,obt_l2_norm,obt_diag_l2_norm,obt_diag_orig_l2_ratio,tbt_l2_norm,tbt_diag_l2_norm,tbt_diag_orig_l2_ratio,diagonal_dominance,diagonal_dominance_calc_time_s
0,"fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'defau...",16,18,32,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,4.714356e+08,8.673422,0.000037,...,12.616499,0.093078,19.050578,45.595818,2.393409,7.245472,4.026223,0.555688,2.949097,0.008731
1,"fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'defau...",39,50,78,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.221223e+21,21.086795,0.000135,...,19.239314,0.011196,27.419685,93.461373,3.408550,12.077686,6.113896,0.506214,3.914764,0.024509
2,fcidump.2_co2_6-311++G__,6,8,12,0,"[0, 0, 0, 0, 0, 0]",1,4.950000e+02,2.694605,0.000439,...,3.070888,0.000323,6.060295,11.192867,1.846918,3.280063,1.907076,0.581415,2.428332,0.000092
3,"fcidump.3_ts_ru_macho_co2_{'Ru'_ 'cc-pVTZ-PP',...",28,34,56,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.142582e+15,15.330938,0.000153,...,14.964989,0.003242,21.145553,63.234225,2.990427,10.116799,5.176423,0.511666,3.502093,0.024683
4,fcidump.5_ts_ru_macho_melact_{'Ru'_ 'cc-pVTZ-P...,25,30,50,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.712921e+13,13.673290,0.000040,...,14.544999,0.002782,20.824048,58.487200,2.808637,9.270778,4.935153,0.532334,3.340972,0.019528
5,fcidump.7_melact_6-311++G__,9,12,18,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1.856400e+04,4.268672,0.000579,...,4.755097,0.000141,8.433882,18.239439,2.162639,5.028479,2.732513,0.543407,2.706046,0.000145
6,fcidump.8_melact_6-311++G__,14,22,28,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,3.767400e+05,5.576042,0.000042,...,5.366092,0.002229,10.641383,30.905188,2.904245,6.159087,3.205206,0.520403,3.424648,0.006821
7,"fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'...",33,46,66,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.066117e+16,16.609180,0.000066,...,6.996984,0.094925,12.856182,62.030199,4.824932,9.723939,4.599275,0.472985,5.297916,0.040287
8,fcidump.10_fecp2+_s0.5_def2-tzvp,46,57,92,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.970221e+25,25.472789,0.000180,...,10.497539,0.175947,17.545480,93.450376,5.326180,13.525605,6.136029,0.453660,5.779840,0.105734
9,fcidump.11_fecp2_s0_def2-tzvp,46,58,92,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.792375e+25,25.253429,0.000150,...,10.662706,0.033005,17.840595,94.482983,5.295955,13.566210,6.173533,0.455067,5.751022,0.083238


In [6]:


datafiles_folder_path = Path(datafiles_folder)
# dpm.load_dmrg_output_data(datafile_path=datafiles_folder_path/datafile_name)
csv_file_list = [f"{file}.csv" for file in fcidump_files]
(
        id_list,
        coarse_bond_dimension_list,
        total_loops_cpu_time_sec_list,
        total_loops_wall_time_sec_list,
    ) = dpm.get_coarse_bond_dimension_needed_many_files(
    csv_file_list=csv_file_list, folder_path=datafiles_folder_path, energy_change_threshold_hartrees=1e-3#1.35e-3#8.2e-3
)
print(id_list, coarse_bond_dimension_list)




# performance_metrics_dictionary = [{"hamiltonian_id":id, "coarse_bond_dimension":coarse_bond_dimension, t} for id, coarse_bond_dimension in zip(id_list, coarse_bond_dimension_list)]
performance_metrics_dictionary = [{"hamiltonian_id":id, "coarse_bond_dimension":coarse_bond_dimension, "total_loops_cpu_time_sec":total_loops_cpu_time_sec, "total_loops_wall_time_sec":total_loops_wall_time_sec} for id, coarse_bond_dimension, total_loops_cpu_time_sec, total_loops_wall_time_sec in zip(id_list, coarse_bond_dimension_list, total_loops_cpu_time_sec_list, total_loops_wall_time_sec_list)]
performance_metrics_df = pd.DataFrame(performance_metrics_dictionary)
# performance_metrics_df
#print shape of dataframe
print(performance_metrics_df.shape)

No energy change less than 0.001 for fcidump.10_fecp2+_s0.5_def2-tzvp.csv
Min Energy change: 0.0013487313265159173
No energy change less than 0.001 for fcidump.54_ts56_{'default' _ '6-31++G(d,p)'}.csv
Min Energy change: 0.0011669683444779366
["fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", "fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", 'fcidump.2_co2_6-311++G__', "fcidump.3_ts_ru_macho_co2_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", "fcidump.5_ts_ru_macho_melact_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}", 'fcidump.7_melact_6-311++G__', 'fcidump.8_melact_6-311++G__', "fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}", 'fcidump.11_fecp2_s0_def2-tzvp', "fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}", "fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.22_pc-_{'Mo'_ 

In [7]:
# # Load hubbard distance results from csv
# import numpy as np

# hubbard_distance_arrays = np.loadtxt(
#     datafiles_folder / Path("nearness_to_hubbard.csv"),
#     dtype=np.str_,
#     delimiter=",",
#     skiprows=0,
# )

# columns = [
#     "hamiltonian_id",
#     "hubbard_dist_obt",
#     "hubbard_dist_tbt",
#     "hubbard_dist_total",
# ]

# # Get correct id name from files
# hubbard_distance_dict_list = []
# name_list = []
# for index, row in enumerate(hubbard_distance_arrays[1:]):
#     mol_id, hubbard_dist_obt, hubbard_dist_tbt, hubbard_dist_total = row
#     fcidump_start = mol_id.split("_")[0]
#     file_name_fci = list(Path(datafiles_folder).glob(f"{fcidump_start}_*"))
#     # print(file_name_fci)
#     if len(file_name_fci) == 1:
#         if file_name_fci[0].name[:-4] in name_list:
#             #Check that the numbers are the same
#             assert hubbard_distance_dict_list[name_list.index(file_name_fci[0].name[:-4])]["hubbard_dist_obt"] == float(hubbard_dist_obt)
#             assert hubbard_distance_dict_list[name_list.index(file_name_fci[0].name[:-4])]["hubbard_dist_tbt"] == float(hubbard_dist_tbt)
#             assert hubbard_distance_dict_list[name_list.index(file_name_fci[0].name[:-4])]["hubbard_dist_total"] == float(hubbard_dist_total)
#             continue
#         # print(f"Found {file_name_fci}")
#         hubbard_distance_dict_list.append(
#             {
#                 "hamiltonian_id": file_name_fci[0].name[:-4],
#                 "hubbard_dist_obt": float(hubbard_dist_obt),
#                 "hubbard_dist_tbt": float(hubbard_dist_tbt),
#                 "hubbard_dist_total": float(hubbard_dist_total),
#             }
#         )
#         name_list.append(file_name_fci[0].name[:-4])
#     elif len(file_name_fci) == 0:
#         print(f"Skipping {mol_id}")
#         continue
#     else:
#         raise ValueError(f"{mol_id},{file_name_fci}")
# hubbard_dist_df = pd.DataFrame(hubbard_distance_dict_list)
# hubbard_dist_df
# print(hubbard_dist_df.shape)
# # print(hubbard_distance_dict_list)
# # temp_dict = {}
# # for fcidump_file in fcidump_files:
# #     mol_index = fcidump_file.split("_")[0].split(".")[1]
# #     temp_dict[mol_index] = fcidump_file

# # fcidump_files_temp = []
# # for key in sorted(temp_dict.keys()):
# #     fcidump_files_temp.append(temp_dict[key])
# # # Put into a list of dictionaries
# # hubbard_distance_dict = [
# #     {
# #         "hamiltonian_id": id,
# #         "hubbard_dist_obt": float(hubbard_dist_obt),
# #         "hubbard_dist_tbt": float(hubbard_dist_tbt),
# #         "hubbard_dist_total": float(hubbard_dist_total),
# #     }
# #     for id, hubbard_dist_obt, hubbard_dist_tbt, hubbard_dist_total in hubbard_distance_arrays[
# #         1:
# #     ]
# # ]
# # # print(hubbard_distance_arrays)
# # hubbard_distance_dict

In [8]:
print(ml_features_df.shape)

(51, 184)


In [9]:
# Combine the dataframes
combined_df = pd.merge(ml_features_df, performance_metrics_df, on="hamiltonian_id",how="inner")
print(combined_df.shape)

(49, 187)


In [10]:
# combined_df = pd.merge(combined_df, hubbard_dist_df, on="hamiltonian_id",how="inner")
# print(combined_df.shape)

In [11]:
combined_df

,hamiltonian_id,num_orbitals,num_electrons,num_spin_orbitals,two_S,orbsym,isym,total_num_states,log10_hilbert_space_size,tbt_matrices_calc_time_s,...,obt_diag_l2_norm,obt_diag_orig_l2_ratio,tbt_l2_norm,tbt_diag_l2_norm,tbt_diag_orig_l2_ratio,diagonal_dominance,diagonal_dominance_calc_time_s,coarse_bond_dimension,total_loops_cpu_time_sec,total_loops_wall_time_sec
0,"fcidump.0_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'defau...",16,18,32,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,4.714356e+08,8.673422,0.000037,...,45.595818,2.393409,7.245472,4.026223,0.555688,2.949097,0.008731,5,217.783149,5.904835
1,"fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'defau...",39,50,78,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.221223e+21,21.086795,0.000135,...,93.461373,3.408550,12.077686,6.113896,0.506214,3.914764,0.024509,15,19333.444100,661.216804
2,fcidump.2_co2_6-311++G__,6,8,12,0,"[0, 0, 0, 0, 0, 0]",1,4.950000e+02,2.694605,0.000439,...,11.192867,1.846918,3.280063,1.907076,0.581415,2.428332,0.000092,5,22.916073,1.152531
3,"fcidump.3_ts_ru_macho_co2_{'Ru'_ 'cc-pVTZ-PP',...",28,34,56,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.142582e+15,15.330938,0.000153,...,63.234225,2.990427,10.116799,5.176423,0.511666,3.502093,0.024683,27,8976.006194,277.678822
4,fcidump.5_ts_ru_macho_melact_{'Ru'_ 'cc-pVTZ-P...,25,30,50,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.712921e+13,13.673290,0.000040,...,58.487200,2.808637,9.270778,4.935153,0.532334,3.340972,0.019528,8,2050.440041,59.459799
5,fcidump.7_melact_6-311++G__,9,12,18,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1.856400e+04,4.268672,0.000579,...,18.239439,2.162639,5.028479,2.732513,0.543407,2.706046,0.000145,9,212.900357,5.578106
6,fcidump.8_melact_6-311++G__,14,22,28,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,3.767400e+05,5.576042,0.000042,...,30.905188,2.904245,6.159087,3.205206,0.520403,3.424648,0.006821,7,343.194229,9.077071
7,"fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'...",33,46,66,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.066117e+16,16.609180,0.000066,...,62.030199,4.824932,9.723939,4.599275,0.472985,5.297916,0.040287,104,35315.029602,1089.647073
8,fcidump.11_fecp2_s0_def2-tzvp,46,58,92,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.792375e+25,25.253429,0.000150,...,94.482983,5.295955,13.566210,6.173533,0.455067,5.751022,0.083238,8,14890.202160,538.292357
9,"fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'...",33,45,66,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,8.906733e+16,16.949718,0.000058,...,61.926301,4.845851,9.798468,4.639426,0.473485,5.319336,0.037281,449,145911.593203,4448.952683


In [12]:
# Save the combined dataframe to csv
combined_df.to_csv("combined_df.csv", index=False)


In [13]:
combined_df[["coarse_bond_dimension"]]

,coarse_bond_dimension
0,5
1,15
2,5
3,27
4,8
5,9
6,7
7,104
8,8
9,449
